# 3D - Conductance in top gated 2DEG

## 1. Introduction
### 1.1 Purpose
With this Jupyter Notebook (JN) you can practice the tutorial [3D - Conductance in top gated 2DEG](https://www.nextnano.com/manual/nextnanoplus_tutorials/electronics/3D_conductance_in_top_gated_2DEG.html). Following this JN you will learn how to extract the potential calculated with **nextnano++** and use it to simulate the transport properties of a simple square QPC in **Kwant**.

### 1.2 What are the engines
This JN uses two independent solvers: **nextnano++** and **Kwant**. The **nextnano++** is a self-consistent Schrödinger-Poisson-Current equations solver for arbitraty 1D, 2D, and 3D semiconductor structures, that can be run either using **nextnanomat** or free **nextnanopy** Python package. **Kwant** is a Python package for numerical calculations on tight-binding models with a strong focus on quantum transport.

### 1.3 What to install
All **nextnano++**, **nextnanopy**, and **Kwant** are used in this notebook. To make sure that you can sucessfully go through this JN, you should make sure that you have istalled the newest versions of the packages and our software. Additionaly, you may be interested in installing **MUMPS** for better computational performance, however, it is not required.

Here is the list of things to install before proceeding further:
- [nextnano Software](https://www.nextnano.com/manual/downloads.html) to simulate an electric potential in 2DEG - the quantum point contact (QPC).
- [nextnanopy Python package](https://github.com/nextnanopy/nextnanopy/blob/master/docs/How%20to%20install%20nextnanopy.md) to run simulations in nextnano++ and read the results using Python script.
- [Kwant Python package](https://kwant-project.org/install) to perform the main simulation, transmission throught the QPC.
- [MUMPS](https://graal.ens-lyon.fr/MUMPS/index.php?page=dwnld) [(conda install)](https://anaconda.org/conda-forge/mumps) to get better performance.

### 1.4 It is just an example - MAKE IT YOURS
This notebook and related input file for **nextnano++** are just an example presenting how to use **nextnano++** and **Kwant** together to use the powerful simulations of quantum transport of **Kwant** for realistic cases modeled by **nextnano++**.

After learning how this JN works:
- You can use this JN as a template and play with the geometry of the QPC to see how it affects the pinch-off curve.
- You can simulate transport properties in other devices, simply by changing the gates geometry in the related input file for **nextnano++**. You can complexify the model used in Kwant by adding more leads, magnetic field, disorder, etc.  It will allow you to model the transport of a 2DEG in whatever structure you wish to consider. ([nextnano++](https://www.nextnano.com/manual/nextnanoplus/index.html), [Kwant](https://kwant-project.org/doc/))
- You can calibrate the **nextnano++** simulation for your exprimental sample to compare the simulation results with measured data ([nextnano++](https://www.nextnano.com/manual/nextnanoplus/index.html))


## 2. Preparing for Simulation

### 2.1 Run nextnano++
This is the moment when you should run the input file **3D_conductance_in_top_gated_2DEG_nnp.in** using the **nextnano++**. It will take a while to get results (maybe even a few hours). The input file is prepared to run 101 simulations for the top-gate biases in the range **from -1.5 to 0.0 V**, and the bottom-gate bias set to **-1.1 V**. The results important for this tutorial will be stored in files named **bandedges_2d_2deg_slice.fld**.

### 2.2 Let's begin with the script
Let's start with importing some packages and defining a few useful constants. Packages for plotting are imported and set up inside the file **matplotlib_setup.ipy**. If the script below runs properly, then you are set up, and everything further should go well.

In [ ]:
import numpy as np         # package with linear algebra (always important)
import kwant as kw         # Kwant - you know about it already! (for quantum transport)

%run matplotlib_setup.ipy  # an *.ipy file importing a few libraries for plotting and setting plots up

import nextnanopy as nn                  # your favourite nextnanopy (to run nextnano ++)
from scipy.interpolate import interp2d   # package with algorithms (here for interpolation only)

e = 1.602e-19          # electron charge (C)
hb = 6.626e-34/2/np.pi # Dirac constant (Js)
h = 6.626e-34          # Planck constant (Js)
nm = 1e-9              # 1 nanometer (m)

### 2.3 Define helping functions
Below, we define a set of functions which will be called later to create the system for simulation of transmission. These functions will allow us to convieniently follow the typical workflow of **Kwant**:
1. creating an “empty” tight binding system,
2. setting its matrix elements and hoppings,
3. attaching leads,
4. and passing the finalized system to the solver.

The first set of functions will be used to define the potential at each node of the thight-binding sysem. They will use potential outputed by **nextnano++** and interpolate it for the lattice defined for **Kwant**. These functions will be called by the solver to obtain the matrix elements.

In [ ]:
def qpc_potential(site): #potential in the scattering region
    x, y = site.pos
    
    return -interpolated_potential(x,y)[0]/T 

def lead_potential(site): #potential in the left lead
    x, y = site.pos
    
    return -interpolated_potential(left,y)[0]/T 


def lead_potential2(site): #potential in the right lead
    x, y = site.pos

    return -interpolated_potential(right,y)[0]/T 


The next functions create the leads, in this case 2D systems with translational symmetry. For the optimisation reasons, they contain their own definitions of functions returning the matrix elements and the hopping parameters (**2nd step**). These functions will call the functions defining the potential of leads passed as **pot**. The leads will represent ohmic contacts and will be further attached to the scattering region. 

In [ ]:
def make_lead_input(pot, t=1): # defineing the right lead

    def onsite(*args): # return the matrix element of a given site (A_i,j)
        return 4 * t - pot(*args)
    
    def hopping(*args): # return the hopping between two given sites (T_i,j,i+1,j)
        return -t 

    syst = kw.Builder(kw.TranslationalSymmetry([-a, 0])) # specifiy a translational symmetry 

    syst[(lat(left//a, y) for y in np.arange(bottom//a+1,top//a,1))] = onsite # define the matrix element of each site 
    syst[lat.neighbors()] = hopping # define the hopping parameters between closest neighbours  
    
    return syst


def make_lead_output(pot, t=1): # defining the left lead
   
    def onsite(*args): # return the matrix element of a given site (A_i,j)
        return 4 * t - pot(*args)
    
    def hopping(*args): # return the hopping between two given sites (T_i,j,i+1,j)
        return -t 

    syst = kw.Builder(kw.TranslationalSymmetry([a, 0])) # specifiy a translational symmetry 
    
    syst[(lat(right//a, y) for y in np.arange(bottom//a+1,top//a,1))] = onsite # define the matrix element of each site
    syst[lat.neighbors()] = hopping # define the hopping parameters between closest neighbours
    
    return syst


The last function defined goes through most of the workflow of **Kwant**. It creates the empty lattice (**1st step**), fills it with the matrix elements and the hopping parameters defining this way the scattering region (**2nd step**), and creates and attaches the leads to the scattering region (**3rd step**). The output of this function is a complete system for simulation in **Kwant**. This function calls our functions defining the potential in the scattering region and in the leads passed as: **pot1**, **pot2**, and **pot3**.

In [ ]:

def make_system(pot1,pot2,pot3, t=1): # define the scattering region
    
    def onsite(*args):
        return 4 * t - pot1(*args)
    
    def hopping(*args):
        return -t

    # Construct the scattering region.
    
    # 1st step - creating an “empty” tight binding system
    sr = kw.Builder()
    
    # 2st step - setting matrix elements
    sr[(lat(x, y) for x in np.arange(left//a+1,right//a,1) for y in np.arange(bottom//a+1,top//a,1))] = onsite
    
    # 2st step - setting hoppings
    sr[lat.neighbors()] = hopping

    # 3rd step - Construct the leads and attach them to the scattering region.
    lead = make_lead_input(pot2, t)
    sr.attach_lead(lead)

    lead3 = make_lead_output(pot3, t)
    sr.attach_lead(lead3)

    return sr

That's all about defining helpful functions. Now it's time to define the actual system (**steps 1-3**) and run the final simulation (**step 4**) within a quite short script.

## 3. Defining the 2D System and Running Kwant

### 3.1 Set some parameters of the system

The script below defines which piece of the electric potential, otput from the nextnano simulation, is taken into account during the simualtion in **Kwant**. It is important to keep in mind that you need to run simulation in nextnano for bigger region than will be passed to Kwant to obtain a realistic potential for QPC. Check that the input file **3D_conductance_in_top_gated_2DEG_nnp.in** generates much larger slices of potential than passed to Kwant. The slices outputed in this tutorial by **nextnano++** are of size 1000 x 600 nm, while to the Kwant we are passing only 400 x 400 nm pieces. The numbers defining size of the pieces of passed potentials are named in the script as: **left**, **right**, **bottom**, and **top**. Their interpretation is analogous to the lines defining regions in **nextnano++**.

The effective mass of the material where the 2DEG is created is chosen.
The lattice constant of the tight-binding system can be arbitrary.

For basic simulations, similar to this tutorial, it may be sufficient for you to modify the heterostructure and geometry of gates in **nextnano++**, and adjust the limits of the simulation defined below. The rest of the python script can be left intact.

In [ ]:
left = -200  # left limit of the scattering region (nm)
right = 200     # right limit of the scattering region (nm)

bottom = -200      # lower limit of the scattering region (nm)
top = 200       # upper limit of the scattering region (nm)

ms = 0.067 * 9.109e-31  # an effective mass of electrons in 2DEG 

a = 1  # lattice constant of the tight-binding system (nm)

T = hb*hb/2/nm/nm/ms/e # constant to convert from eV (output of nn++) to kwant energy unit

### 3.2 Let's check what is modeled

In this part the Kwant workflow begins. The goal of the script below is to show the lattice defined for simulation in **Kwant** (**1st plot**), potential energy which is passed to the simulation (**2nd plot**), and the electronic band structure of the leads (**3rd plot**). The simulation here is held for the binding constant set to $1/a^{2}$ so that you can change the lattice constant without changing the energy unit in the **Kwant**.

To make this part of the script working, you need to change the path to the **bandedges_2d_2deg_slice.fld** containing potential from one of your simulations in **nextnano++**, as the one present in the script is just an exemplary one.

In [ ]:
# +++++ extract and interpolate the potential from nextnano +++++ #
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #

# Opening the output file with the potential
# Here you have to indicate the path to your nn++ output file: bandedges_2d_2deg_slice.fld

# EDIT THE PATH TO YOUR OUTPUT FILE
# EDIT THE PATH TO YOUR OUTPUT FILE
# EDIT THE PATH TO YOUR OUTPUT FILE
extracted_potential = nn.DataFile(r'C:\...\3D_conductance_in_top_gated_2DEG_nnp\bias_00000\bandedges_2d_2deg_slice.fld',product='nextnano++')
# EDIT THE PATH TO YOUR OUTPUT FILE
# EDIT THE PATH TO YOUR OUTPUT FILE
# EDIT THE PATH TO YOUR OUTPUT FILE

# Extracting the data using nextnanopy
x = extracted_potential.coords['x'].value
y = extracted_potential.coords['y'].value
potential = extracted_potential.variables['Gamma'].value

# Interpolating the extracted electric potential to fit the lattice passed to Kwant
# It is used by some of the functions defined before - the first set
interpolated_potential=interp2d(x,y,potential.T) # HOW THIS POTENTIAL ENTER THE QPC?

# +++++ define the system (by calling the previously defined functions) +++++ #
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #

# Creating a square lattice 
lat = kw.lattice.square(a) 

# Defining the tight binding model (scattering region and leads)
# These are the first three steps of the Kwant workflow called in one go
qpc = make_system(qpc_potential,lead_potential,lead_potential2,t=1/a**2) 

# +++++ Plot what is defined +++++ #
# ++++++++++++++++++++++++++++++++ #

# A plot of the system - the lattice of the scattering region with leads visible as red lines on the lef and right border
kw.plot(qpc); 

# A plot the electric potential that will be passed to the Kwant in this short piece of script
kw.plotter.map(qpc, lambda s: qpc_potential(s,)); 

# +++++ Get really ready to run the simulation - finalization of the structure +++++ #
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #

# Getting ready for the simulation
fqpc = qpc.finalized()

# A plot of the electronic band strucure of the first lead, the one on the left hand side (both are the same)
kw.plotter.bands(fqpc.leads[0]);

Now, after you have verified lattice, potential and lead bands entering the simulation, it is finally the time for the **4th** step of the **Kwant** workflow - passing the finalized system to the solver (and getting some results). Here, the direct output of the **Kwant** is an object **smatrix** containing all the scattering matrices. You have an easy access to the transmission conductnace between leads in the system using this object. You should get transmission equal to **5.7658...**

In [ ]:
# +++++ Get ready and run the simulation (the 4th step) +++++ #
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #

# Solving the scattering problem in Kwant
# It will calculute the transmissions from lead 0 to others (here 0 and 1) 
# at energy 0 which correpsonds to the Fermi level defined in nextnano
smatrix = kw.smatrix(fqpc, 0, in_leads=[0]) 

smatrix.transmission(1,0) #transmission conductance from lead 0 to lead 1 (in unit of 2e^2/h)

### 3.3 Plot of transmission vs voltage - the final simulation
In this section we actually repeate again everything what you did in the section **3.2**. This time, however, the simulation is performet for all 101 potentials calculated in **nextnano++**. The pinch of curve is plotted as the a final result of this piece of the script.

Here as well, you need to refine the path to your output files (the **line 17**).

In [ ]:
# +++++ Paramteres defining number of output files generated in nextnano++ and related voltages +++++ #
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
N = 101 # number of input files generated by nextnano++ to create the pinch-off curve
Gate_Volt_min = -2.2 # voltage onthe gate for the first element of the sweep
Gate_Volt_max = 0.0 # voltage on the gate for the last element of the sweep

# +++++ Reading, Preparing, and Simulationg +++++ #
# +++++++++++++++++++++++++++++++++++++++++++++++ #
transmission = []

print('Iteration:', end = ' ')
for k in range(N):
    print(str(k), end = ' ')
    
    # reading 

    # EDIT THE PATH TO YOUR OUTPUT FILE
    # EDIT THE PATH TO YOUR OUTPUT FILE
    # EDIT THE PATH TO YOUR OUTPUT FILE
    extracted_potential = nn.DataFile(fr'C:\...\3D_conductance_in_top_gated_2DEG_nnp\bias_{str(k).zfill(5)}\bandedges_2d_2deg_slice.fld',product='nextnano++')
    # EDIT THE PATH TO YOUR OUTPUT FILE
    # EDIT THE PATH TO YOUR OUTPUT FILE
    # EDIT THE PATH TO YOUR OUTPUT FILE
    
    x = extracted_potential.coords['x'].value
    y = extracted_potential.coords['y'].value
    potential = extracted_potential.variables['Gamma'].value

    # preparing
    
    interpolated_potential=interp2d(x, y, potential.T)
    lat = kw.lattice.square(a)
    qpc = make_system(qpc_potential, lead_potential, lead_potential2, t=1/a**2)
    fqpc = qpc.finalized()
    
    # simulating
    
    smatrix = kw.smatrix(fqpc, 0, in_leads = [0])
    transmission.append(smatrix.transmission(1,0))
    
# +++++ Plotting +++++ #
# ++++++++++++++++++++ #

V=np.linspace(Gate_Volt_min, Gate_Volt_max, N)
plt.plot(V, transmission, color = 'red')
plt.xlabel('upper gate voltage (V)')
plt.ylabel('Conductance (2e^2/h)')
plt.title('Transmission between input lead and output lead\n')
